<a href="https://colab.research.google.com/github/microprediction/endersnotebooks/blob/main/mean_reversion_attacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade git+https://github.com/microprediction/endersgame.git

  Cloning https://github.com/microprediction/endersgame.git to /tmp/pip-req-build-4310jkky
  Running command git clone --filter=blob:none --quiet https://github.com/microprediction/endersgame.git /tmp/pip-req-build-4310jkky
  Resolved https://github.com/microprediction/endersgame.git to commit fc21440aaaa6fb2ab3282964a32bb83a2e9e2c58
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.3 MB/s eta 0:00:00
  Created wheel for endersgame: filename=endersgame-0.3.2-py3-none-any.whl size=28574 sha256=cab08c6088a0ac4581982ec238e50c012415e13331f9be22539f7dada9896eb0
  Stored in directory: /tmp/pip-ephem-wheel-cache-kmf8l90i/wheels/39/24/f0/19aeef5765f9b9f629bab092893ebd3c04bde902d978c742bb
Successfully built endersgame


# Mean Reversion Attacker Tutorial
This notebook demonstrates how to create an "attacker" (see [README.md](https://github.com/microprediction/endersgame/tree/main/endersgame/attackers)), and test it.

We use the steam generator (see this [notebook](https://github.com/microprediction/endersnotebooks/blob/main/enders_data_generator.ipynb)) to train it.  

## What should an attacker do?

It tries to predict `up` or `down` but not too often.

Our attacker will consume a univariate sequence of numerical data points $x_1, x_2, \dots x_t$ and try to exploit deviations from the [martingale property](https://en.wikipedia.org/wiki/Martingale_(probability_theory)), which is to say that we expect the series $x_t$ to satisfy:

$$ E[x_{t+k}] \approx x_t $$

roughly. Of course, there's no such thing in this world as a perfect martingale and it is your job to indicate when

$$ E[x_{t+k}] > x_t $$

or conversely.

## Overview
We will


1.   Start with an attacker that already has some accounting logic
2.   Modify the default `tick` and `predict` methods
3.   Run the attacker on mock data
4.   Run the attacker on real data
5.   Set up an optimization to tune the attacker's parameters
6.   See if it helps on the test set


## Imports


In [38]:
from endersgame.attackers.attackerwithsimplepnl import AttackerWithSimplePnL
from endersgame.rivertransformers.macd import MACD
from endersgame.datasources.streamgenerator import stream_generator
from river import stats
import numpy as np
import math
import types
from pprint import pprint
import json
import scipy.optimize as opt
from endersgame.datasources.streamgeneratorgenerator import stream_generator_generator
from endersgame.gameconfig import HORIZON
from typing import Iterable, Iterator



## Step 1: Decide what state to maintain
Let's first implement the `tick` method. This should quickly respond to an incoming data point by modifying a rapidly changing `state`. Here we choose to maintain the current value and also an exponentially weighted moving average of historical values. We use a simple dictionary, but other styles are presented [here](https://github.com/microprediction/endersgame/blob/main/tests/colabexamples/README.md) that you might prefer.

In [18]:
from endersgame.attackers.attackerwithsimplepnl import AttackerWithSimplePnL

class MyAttacker(AttackerWithSimplePnL):

     def __init__(self, a=0.01):
        super().__init__()
        self.state = {'running_avg':None,
                      'current_value':None}
        self.params = {'a':a}

     def tick(self, x:float):
         # Maintains an expon moving average of the data
         self.state['current_value'] = x
         if not np.isnan(x):
            if self.state['running_avg'] is None:
                self.state['running_avg'] = x
            else:
                self.state['running_avg'] = (1-self.params['a'])*self.state['running_avg'] + self.params['a']*x


### Testing tick
We are half way there. Let's check the state maintenance:

In [19]:
x_train_stream = stream_generator(stream_id=0,category='train')
attacker = MyAttacker()
for x in x_train_stream:
    attacker.tick(x)

print(f"After processing the entire stream, the current value is  {attacker.state['current_value']} and the moving average is {attacker.state['running_avg']}")
attacker.state

After processing the entire stream, the current value is  10319.115384618284 and the moving average is 10317.31274292457


{'running_avg': 10317.31274292457, 'current_value': 10319.115384618284}

## Making an `up` or `down` decision
Next we implement `predict` using a mean reversion strategy.

In [20]:
def predict(self, horizon:int=None)->float:
    if self.state['current_value'] > self.state['running_avg'] + 2:
        return -1
    if self.state['current_value'] < self.state['running_avg'] - 2:
        return 1
    return 0

attacker = MyAttacker()
attacker.predict = types.MethodType(predict, attacker) # <-- Attach the predict method to our existing instance of attacker


(By the way you'll notice that we're constructing the class incrementally by assigning the method manually, just to allow this notebook to flow. This will look unnatural to many of you. By all means just move the predict definition back into the actual class.)

Anyway, let's check that if the current value is very high we should predict it will fall:

In [21]:
attacker.state['current_value'] = 10
attacker.state['running_avg'] = 5
print(attacker.predict())

-1


## Run the attacker on mock data
Let's put these together to creat an attacker with both `tick` and `predict`

In [22]:
attacker = MyAttacker()               # Always reset an attacker
attacker.predict = types.MethodType(predict, attacker)  # <-- Again, if you find this awkward, you can always just put predict() in the class itself.

xs = [1,3,4,2,4,5,1,5,2,5,10]*100
for x in xs:
   y = attacker.tick_and_predict(x=x, horizon=HORIZON)

## Run the attacker on real data

In [23]:
x_test_stream = stream_generator(stream_id=1,category='train')
attacker = MyAttacker()
attacker.predict = types.MethodType(predict, attacker)     #  <-- If you get sick of doing this then put the method in the class at the outset
for x in x_test_stream:
    y = attacker.tick_and_predict(x=x,horizon=HORIZON)

attacker.state

{'running_avg': 10242.47896425911, 'current_value': 10242.923076925954}

## Check the attacker's profit and loss


In [24]:
pprint(attacker.pnl.summary())

{'current_ndx': 10232,
 'losses': 36,
 'num_resolved_decisions': 62,
 'profit_per_decision': -0.19052109181169186,
 'standardized_profit_per_decision': -0.10475745311387367,
 'total_profit': -11.812307692324895,
 'win_loss_ratio': 0.7222222222222222,
 'wins': 26}


## Train (globally) the using many streams
Let's create a function that evaluates the attacker for a choice of parameter `a` when it is run over the entire training set. For this you can use the stream generator generator.



In [35]:
# First define the objective as negative total profit and test it


def negative_attacker_profit(a, streams: Iterable[Iterable[float]], verbose=True):
    """

         a:          Parameter
         streams:    Supplies a collection of individual streams

    """
    total_profit = 0

    for stream_gen in streams:

        # Reset the attacker each stream
        attacker = MyAttacker(a=a)
        attacker.predict = types.MethodType(predict, attacker)

        # Run it over the stream
        for x in stream_gen:
            y = attacker.tick_and_predict(x=x,horizon=HORIZON)

        # Accumulate the profit or loss
        pnl = attacker.pnl.summary()
        total_profit += pnl['total_profit']

    if verbose:
        print(f'Using a={a} the total profit is {total_profit}')
    return -total_profit         # So smaller is better for the optimizer


negative_attacker_profit(a=0.1,streams=stream_generator_generator(category='train'))

Using a=0.1 the total profit is -69.96167552893994


69.96167552893994

### The `train` function
The canonical way to write a training procedure uses `streams` argument and iterates over all data points in all training streams.

In [69]:
from typing import Iterable
import json
import os

def local_model_directory_path():
     dr = os.path.join('local_model_path')
     os.makedirs(dr, exist_ok=True)
     return dr


def train(
             streams: Iterable[Iterable[float]],
             model_directory_path: str
          ):

    def training_optimization_objective(a):
        return negative_attacker_profit(a=0.06, streams=streams)

    result = opt.minimize_scalar(training_optimization_objective, bounds=(0.001, 0.2), method='bounded',options={'maxiter': 5})

    print(f"Optimal value of a: {result.x}")
    print(f"Minimum total profit: {-result.fun}")  # Re-negate to get the actual profit

    # Let's save the best parameter
    best_a = result.x
    parameter_file = os.path.join(model_directory_path,'params.json')
    with open(parameter_file,'w') as fp:
        json.dump({'a':best_a}, fp)
        print(f'Saved {parameter_file}')

    # Check we can load it again!
    with open(parameter_file,'r') as fp:
        params = json.load(fp)
        print(params)


# Here is how you would use it on the training data
best_a = train(streams=stream_generator_generator(category='train'), model_directory_path=local_model_directory_path())

Using a=0.06 the total profit is -100.81465566355791
Using a=0.06 the total profit is 0
Using a=0.06 the total profit is 0
Using a=0.06 the total profit is 0
Using a=0.06 the total profit is 0
Optimal value of a: 0.13296067316430177
Minimum total profit: 0
Saved local_model_path/params.json
{'a': 0.13296067316430177}


In [ ]:
#def train(
#             streams: Iterable[Iterable[float]]
#          ):
#    pass

## The `infer` function
Your notebook should have an infer function that can yield one prediction at a time



In [72]:
def infer(
    stream: Iterator[float],
    model_directory_path: str
):

    # Load the best parameters
    parameter_file = os.path.join(model_directory_path,'params.json')
    with open(parameter_file,'r') as fp:
        params = json.load(fp)
    a = params['a']

    # Instantiate your attacker
    attacker = MyAttacker(a=a)
    yield  # <---  Leave this here. This signals to the system that your attacker is initialized and ready.

    for x in stream:
       decision = attacker.tick_and_predict(x)
       yield decision          # <---- Be sure to yield, even if the decision is zero.



# A quick test that indicates how your infer function will be used when you upload this notebook:
xs = np.random.randn(5)
for y in infer(xs, model_directory_path=local_model_directory_path()):
    print(y)

None
0
0
0
0
0


Not that as far as the API for the contest is concerned, your `infer` could also be much simpler if you wish. It doesn't need to use an attacker at all. For example...  

In [ ]:
def infer(
    stream: Iterator[float]
):
    yield     # <-- I'm ready!
    for x in stream:
        if x>3:
            yield 1
        yield 0


